# Queries

In [1]:
import pandas as pd 
import sqlite3


def create_df(cursor: sqlite3.Cursor) -> pd.DataFrame:
    data = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    df = pd.DataFrame(data, columns=column_names)
    return df


conn = sqlite3.connect(":memory:")

#### Prepare Database

In [2]:
st = """ 
    CREATE TABLE nomnom (
        name TEXT,
        neighborhood TEXT,
        cuisine TEXT,
        review INTEGER,
        price TEXT,
        health TEXT
    );
"""
conn.execute(st)

In [3]:
df = pd.read_csv("nomnom.csv")


names = df["name"].tolist()
neighborhoods = df["neighborhood"].tolist()
cuisines = df["cuisine"].tolist()
reviews = df["review"].tolist()
prices = df["price"].tolist()
healths = df["health"].tolist()

In [4]:
conn.executemany(
    """INSERT INTO nomnom (name, neighborhood, cuisine, review, price, health) 
        VALUES (?, ?, ?, ?, ?, ?);
    """,
    zip(names, neighborhoods, cuisines, reviews, prices, healths)
)

#### 1.

Start by getting a feel for the `nomnom` table:

```sql
SELECT *
FROM nomnom;
```

What are the column names?

In [5]:
df = create_df(conn.execute("SELECT * FROM nomnom;"))
df.head()

,name,neighborhood,cuisine,review,price,health
0,Peter Luger Steak House,Brooklyn,Steak,4.4,$$$$,A
1,Jongro BBQ,Midtown,Korean,4.5,$$,A
2,Pocha 32,Midtown,Korean,4.0,$$,A
3,Nom Wah Tea Parlor,Chinatown,Chinese,4.2,$,A
4,Roberta's,Brooklyn,Pizza,4.4,$$,A


#### 2.

What are the distinct `neighborhood`s?

In [6]:
st = """
    SELECT DISTINCT neighborhood 
    FROM nomnom;
"""
create_df(conn.execute(st))

,neighborhood
0,Brooklyn
1,Midtown
2,Chinatown
3,Uptown
4,Queens
5,Downtown


#### 3.

What are the distinct `cuisine` types?

In [7]:
st = """
    SELECT DISTINCT cuisine 
    FROM nomnom;
"""
create_df(conn.execute(st))

,cuisine
0,Steak
1,Korean
2,Chinese
3,Pizza
4,Ethiopian
5,Vegetarian
6,Italian
7,Japanese
8,American
9,Mediterranean


#### 4.

Suppose we would like some `Chinese` takeout.

What are our options?

In [8]:
st = """
    SELECT * 
    FROM nomnom
    WHERE cuisine = 'Chinese';
"""
create_df(conn.execute(st))

,name,neighborhood,cuisine,review,price,health
0,Nom Wah Tea Parlor,Chinatown,Chinese,4.2,$,A
1,Nan Xiang Xiao Long Bao,Queens,Chinese,4.2,$,A
2,Mission Chinese Food,Downtown,Chinese,3.9,$$,A
3,Baohaus,Downtown,Chinese,4.2,$,A
4,Xi'an Famous Foods,Midtown,Chinese,4.4,$,A
5,Sonnyboy's,Brooklyn,Chinese,4.2,$$,A
6,Great NY Noodletown,Chinatown,Chinese,4.1,$$,B
7,Golden Unicorn,Chinatown,Chinese,3.8,$$,A
8,Wo Hop,Chinatown,Chinese,4.2,$$,None
9,Ping's Seafood,Chinatown,Chinese,4.2,$$,A


#### 5.

Return all the restaurants with `reviews` of 4 and above.

In [9]:
st = """
    SELECT * 
    FROM nomnom
    WHERE review >= 4;
"""
create_df(conn.execute(st)).head()

,name,neighborhood,cuisine,review,price,health
0,Peter Luger Steak House,Brooklyn,Steak,4.4,$$$$,A
1,Jongro BBQ,Midtown,Korean,4.5,$$,A
2,Pocha 32,Midtown,Korean,4.0,$$,A
3,Nom Wah Tea Parlor,Chinatown,Chinese,4.2,$,A
4,Roberta's,Brooklyn,Pizza,4.4,$$,A


#### 6.

Suppose Abbi and Ilana want to have a fancy dinner date.

Return all the restaurants that are `Italian` and `$$$.`

In [10]:
st = """
    SELECT * 
    FROM nomnom
    WHERE cuisine = 'Italian' 
        AND price = '$$$';
"""
create_df(conn.execute(st))

,name,neighborhood,cuisine,review,price,health
0,al di la Trattoria,Brooklyn,Italian,4.4,$$$,A
1,Locanda Vini & Olii,Brooklyn,Italian,4.4,$$$,A
2,Rao's,Uptown,Italian,4.2,$$$,A
3,Carbone,Downtown,Italian,4.3,$$$,A
4,I Sodi,Downtown,Italian,4.5,$$$,A
5,Lilia,Brooklyn,Italian,4.4,$$$,A


#### 7.

Your coworker Trey can’t remember the exact name of a restaurant he went to but he knows it *contains* the word ‘meatball’ in it.

Can you find it for him using a query?

In [11]:
st = """
    SELECT * 
    FROM nomnom
    WHERE name 
        LIKE '%meatball%';
"""
create_df(conn.execute(st))

,name,neighborhood,cuisine,review,price,health
0,The Meatball Shop,Downtown,American,4.2,$,A


#### 8.

Let’s order delivery to the house!

Find all the close by spots in `Midtown`, `Downtown` or `Chinatown`.

(OR can be used more than once)

In [12]:
st = """
    SELECT *
    FROM nomnom
    WHERE neighborhood = 'Midtown' 
        OR neighborhood = 'Downtown' 
        OR neighborhood = 'Chinatown'; 
"""
create_df(conn.execute(st)).head()

,name,neighborhood,cuisine,review,price,health
0,Jongro BBQ,Midtown,Korean,4.5,$$,A
1,Pocha 32,Midtown,Korean,4.0,$$,A
2,Nom Wah Tea Parlor,Chinatown,Chinese,4.2,$,A
3,Buddha Bodai,Chinatown,Vegetarian,4.2,$$,A
4,Mission Chinese Food,Downtown,Chinese,3.9,$$,A


#### 9.

Find all the `health` grade pending restaurants (empty values).

In [13]:
st = """
    SELECT *
    FROM nomnom
    WHERE health IS NULL; 
"""
create_df(conn.execute(st))

,name,neighborhood,cuisine,review,price,health
0,Massawa,Uptown,Ethiopian,4.0,$$,None
1,Wo Hop,Chinatown,Chinese,4.2,$$,None
2,Jacob's Pickles,Uptown,American,NaN,$$,None
3,Lighthouse,Brooklyn,American,4.6,$$,None
4,Los Hermanos,Brooklyn,Mexican,4.4,$,None


#### 10.

Create a Top 10 Restaurants Ranking based on `review`s.

In [14]:
st = """
    SELECT *
    FROM nomnom
    ORDER BY review DESC
    LIMIT 10; 
"""
create_df(conn.execute(st))

,name,neighborhood,cuisine,review,price,health
0,Bunna Cafe,Brooklyn,Ethiopian,4.6,$$,A
1,Russ & Daughters,Downtown,American,4.6,$$,A
2,Lighthouse,Brooklyn,American,4.6,$$,None
3,Jongro BBQ,Midtown,Korean,4.5,$$,A
4,Dan and John's Wings,Downtown,American,4.5,$,A
5,I Sodi,Downtown,Italian,4.5,$$$,A
6,Jajaja,Downtown,Vegetarian,4.5,$$,A
7,St. Anselm,Brooklyn,Steak,4.5,$$,A
8,Marea,Midtown,Italian,4.5,$$$$,A
9,Ootoya,Downtown,Japanese,4.5,$$,A


#### 11.

Use a `CASE` statement to change the rating system to:

- `review > 4.5` is Extraordinary
- `review > 4` is Excellent
- `review > 3` is Good
- `review > 2` is Fair
- Everything else is Poor

Don’t forget to rename the new column!

In [ ]:
st = """
    SELECT name,
    CASE
        WHEN review > 4.5 THEN 'Extraordinary'
        WHEN review > 4 THEN 'Excellent'
        WHEN review > 3 THEN 'Good'
        WHEN review > 2 THEN 'Fair'
        ELSE 'Poor'
    END AS 'Review'
    FROM nomnom;
"""
create_df(conn.execute(st)).head()

,name,Review
0,Peter Luger Steak House,Excellent
1,Jongro BBQ,Excellent
2,Pocha 32,Good
3,Nom Wah Tea Parlor,Excellent
4,Roberta's,Excellent
